In [2]:
# @title 1. Setup and Installation
print("Starting the installation of torchreid and its dependencies...")
!git clone https://github.com/VlSomers/keypoint_promptable_reidentification.git --quiet

import os
file_to_patch = '/content/keypoint_promptable_reidentification/torchreid/data/datasets/image/occluded_posetrack21.py'

with open(file_to_patch, 'r') as f:
    lines = f.readlines()

future_import_line_index = -1
for i, line in enumerate(lines):
    if 'from __future__ import' in line:
        future_import_line_index = i
        break

if future_import_line_index != -1:
    lines.insert(future_import_line_index + 1, 'from dataclasses import field\n')

patched_content = "".join(lines)
patched_content = patched_content.replace('image_gt: pd.DataFrame = pd.DataFrame()', 'image_gt: pd.DataFrame = field(default_factory=pd.DataFrame)')
patched_content = patched_content.replace('image_detections: pd.DataFrame = pd.DataFrame()', 'image_detections: pd.DataFrame = field(default_factory=pd.DataFrame)')

with open(file_to_patch, 'w') as f:
    f.write(patched_content)

print("✅ Patched torchreid library for pandas compatibility.")

%cd keypoint_promptable_reidentification
!pip install -r requirements.txt --quiet
!pip install ultralytics opencv-python-headless scikit-learn numpy tqdm pillow 'scenedetect[opencv]' filterpy --quiet
!python setup.py develop --quiet
print("\n✅ Installation complete!")

try:
    import torch
    import torchreid
    print(f"PyTorch version: {torch.__version__}")
    print(f"Torchreid version: {torchreid.__version__}")
    print(f"CUDA available: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"GPU: {torch.cuda.get_device_name(0)}")
except ImportError as e:
    print(f"⚠️ Error importing libraries: {e}. Please check the installation.")

print("\nDownloading the pre-trained KPR model...")
!wget -P pretrained/ https://github.com/VlSomers/keypoint_promptable_reidentification/releases/download/v1.0/kpr_solider_occ_duke.pth --quiet
print("✅ KPR model downloaded successfully!")

%cd ..
os.makedirs('/content/videos', exist_ok=True)
os.makedirs('/content/output', exist_ok=True)
os.makedirs('/content/temp_clips', exist_ok=True)
print("\nSetup complete! You can now upload a video in the next cell.")

Starting the installation of torchreid and its dependencies...
✅ Patched torchreid library for pandas compatibility.
/content/keypoint_promptable_reidentification
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 4.0 MB/s eta 0:00:00
Reason for being yanked: <none given>
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.7/125.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 46.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.2/256.2 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.2/94.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.1/91.1 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 82.8 MB/s eta 0:00:00
   ━━━

ValueError: mutable default <class 'pandas.core.frame.DataFrame'> for field image_gt is not allowed: use default_factory

In [3]:
# @title 2. Upload Video
from google.colab import files
import shutil
import os

print("Please upload your soccer match video file.")
uploaded = files.upload()

video_path = None
for filename in uploaded.keys():
    if filename.lower().endswith(('.mp4', '.avi', '.mov')):
        # Check if the file exists in the current directory before moving
        if os.path.exists(filename):
            destination_path = f'/content/videos/{filename}'
            shutil.move(filename, destination_path)
            print(f"✅ Video uploaded successfully: {destination_path}")
            video_path = destination_path
            break
        else:
            print(f"⚠️ Uploaded file '{filename}' not found in the current directory.")


if not video_path:
    print("⚠️ No video file found or uploaded file not accessible. Please upload an MP4, AVI, or MOV file.")

Please upload your soccer match video file.


Saving 9.mp4 to 9.mp4


FileNotFoundError: [Errno 2] No such file or directory: '/content/videos/9.mp4'

In [4]:
# @title 3. Imports and Class Definitions
import cv2
import torch
import numpy as np
import json
import pandas as pd
from ultralytics import YOLO
from tqdm.notebook import tqdm
import subprocess
from scipy.optimize import linear_sum_assignment
import torch.nn.functional as F
from typing import List, Dict
from filterpy.kalman import KalmanFilter
from collections import defaultdict
import sys
import os

sys.path.append('/content/keypoint_promptable_reidentification')
from torchreid.models import build_model
from torchreid.utils import load_pretrained_weights
from main import build_config
from torchreid.data.transforms import build_transforms

class SoccerPlayerDetector:
    def __init__(self, model_name: str = 'yolov8x.pt', conf_thresh: float = 0.3):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model = YOLO(model_name)
        self.model.to(self.device)
        self.conf_thresh = conf_thresh
        print(f"Detector initialized on {self.device} with SOTA model {model_name}")

    def process_video(self, video_path: str, output_path: str) -> List[Dict]:
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened(): return []
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        all_detections = []
        with tqdm(total=total_frames, desc="Stage 1: Detecting players") as pbar:
            for frame_idx in range(total_frames):
                ret, frame = cap.read()
                if not ret: break
                results = self.model(frame, classes=[0], imgsz=1280, verbose=False)
                detections = []
                if len(results) > 0 and results[0].boxes is not None:
                    for box in results[0].boxes:
                        if box.conf[0] >= self.conf_thresh:
                            x1, y1, x2, y2 = [int(c) for c in box.xyxy[0].tolist()]
                            detections.append({'bbox': [x1, y1, x2, y2], 'confidence': float(box.conf[0])})
                all_detections.append({"frame_id": frame_idx, "detections": detections})
                pbar.update(1)
        cap.release()
        with open(output_path, 'w') as f: json.dump(all_detections, f, indent=2)
        return all_detections

class STrack:
    def __init__(self, tlwh, score):
        self._tlwh = np.asarray(tlwh, dtype=np.float32)
        self.score = score
        self.track_id = 0
        self.state = 'new'
        self.is_activated = False
        self.frame_id = 0
        self.start_frame = 0
        self.time_since_update = 0
        self.kalman_filter = self.init_kalman_filter()
        initial_state = self.tlwh_to_xyah(self._tlwh)
        self.mean = np.array([initial_state[0], initial_state[1], initial_state[2], initial_state[3], 0, 0, 0, 0], dtype=np.float32)
        self.covariance = np.eye(8) * 10

    def init_kalman_filter(self):
        kf = KalmanFilter(dim_x=8, dim_z=4)
        kf.F = np.array([[1,0,0,0,1,0,0,0],[0,1,0,0,0,1,0,0],[0,0,1,0,0,0,1,0],[0,0,0,1,0,0,0,1],
                         [0,0,0,0,1,0,0,0],[0,0,0,0,0,1,0,0],[0,0,0,0,0,0,1,0],[0,0,0,0,0,0,0,1]], dtype=np.float32)
        kf.H = np.array([[1,0,0,0,0,0,0,0],[0,1,0,0,0,0,0,0],[0,0,1,0,0,0,0,0],[0,0,0,1,0,0,0,0]], dtype=np.float32)
        kf.R[2:, 2:] *= 10.
        kf.P[4:, 4:] *= 1000.
        kf.P *= 10.
        kf.Q[-1, -1] *= 0.01
        kf.Q[4:, 4:] *= 0.01
        return kf

    def tlwh_to_xyah(self, tlwh):
        ret = np.asarray(tlwh).copy()
        ret[:2] += ret[2:] / 2
        if ret[3] > 0: ret[2] /= ret[3]
        return ret

    def xyah_to_tlwh(self, xyah):
        ret = np.asarray(xyah).copy()
        ret[2] *= ret[3]
        ret[:2] -= ret[2:] / 2
        return ret

    def predict(self):
        self.mean = self.kalman_filter.F @ self.mean
        self.covariance = self.kalman_filter.F @ self.covariance @ self.kalman_filter.F.T + self.kalman_filter.Q
        self._tlwh = self.xyah_to_tlwh(self.mean[:4])

    def update(self, detection):
        measurement = self.tlwh_to_xyah(detection['bbox'])
        self.score = detection['confidence']
        y = measurement - self.kalman_filter.H @ self.mean
        S = self.kalman_filter.H @ self.covariance @ self.kalman_filter.H.T + self.kalman_filter.R
        K = self.covariance @ self.kalman_filter.H.T @ np.linalg.inv(S)
        self.mean = self.mean + K @ y
        self.covariance = (np.eye(8) - K @ self.kalman_filter.H) @ self.covariance
        self._tlwh = self.xyah_to_tlwh(self.mean[:4])
        self.state = 'tracked'
        self.is_activated = True
        self.time_since_update = 0

    def activate(self, frame_id, track_id):
        self.track_id = track_id
        self.frame_id = frame_id
        self.start_frame = frame_id
        self.state = 'tracked'
        self.is_activated = True

    @property
    def tlwh(self): return self._tlwh.copy()
    @property
    def tlbr(self):
        ret = self._tlwh.copy()
        ret[2:] += ret[:2]
        return ret

class ByteTrack:
    def __init__(self, high_thresh: float = 0.6, low_thresh: float = 0.1, max_time_lost: int = 90):
        self.tracked_stracks, self.lost_stracks, self.removed_stracks = [], [], []
        self.frame_id, self.track_id_count = 0, 0
        self.high_thresh, self.low_thresh, self.max_time_lost = high_thresh, low_thresh, max_time_lost

    def update(self, detections: List[Dict]) -> List[Dict]:
        self.frame_id += 1
        activated_starcks, lost_stracks, removed_stracks = [], [], []
        dets_high = [d for d in detections if d['confidence'] >= self.high_thresh]
        dets_low = [d for d in detections if self.low_thresh <= d['confidence'] < self.high_thresh]
        stracks_high = [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_high]
        for s in self.tracked_stracks: s.predict()
        dists = self.iou_distance(self.tracked_stracks, stracks_high)
        matches, u_track, u_detection = self.linear_assignment(dists, 0.8)
        for i, j in matches:
            track, det = self.tracked_stracks[i], stracks_high[j]
            track.update({'bbox': det.tlwh, 'confidence': det.score})
            activated_starcks.append(track)
        unmatched_tracks = [self.tracked_stracks[i] for i in u_track]
        dists = self.iou_distance(unmatched_tracks, [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_low])
        matches, u_track, _ = self.linear_assignment(dists, 0.5)
        for i, j in matches:
            track, det = unmatched_tracks[i], [STrack([*d['bbox'][:2], d['bbox'][2]-d['bbox'][0], d['bbox'][3]-d['bbox'][1]], d['confidence']) for d in dets_low][j]
            track.update({'bbox': det.tlwh, 'confidence': det.score})
            activated_starcks.append(track)
        for i in u_track:
            track = unmatched_tracks[i]
            track.time_since_update += 1
            if track.time_since_update > self.max_time_lost: track.state = 'removed'; removed_stracks.append(track)
            else: track.state = 'lost'; lost_stracks.append(track)
        for i in u_detection:
            track = stracks_high[i]
            if track.score >= self.high_thresh:
                self.track_id_count += 1
                track.activate(self.frame_id, self.track_id_count)
                activated_starcks.append(track)
        self.tracked_stracks = [t for t in self.tracked_stracks if t.state == 'tracked'] + activated_starcks
        self.lost_stracks = [t for t in self.lost_stracks if t.time_since_update <= self.max_time_lost] + lost_stracks
        self.removed_stracks.extend(removed_stracks)
        return [{'track_id': t.track_id, 'bbox': [int(x) for x in t.tlbr]} for t in self.tracked_stracks if t.is_activated]

    def iou_distance(self, atracks: List[STrack], btracks: List[STrack]) -> np.ndarray:
        if not atracks or not btracks: return np.empty((len(atracks), len(btracks)))
        atlbrs = np.array([track.tlbr for track in atracks])
        btlbrs = np.array([track.tlbr for track in btracks])
        ious = np.zeros((len(atlbrs), len(btlbrs)), dtype=float)
        for i, a in enumerate(atlbrs):
            for j, b in enumerate(btlbrs):
                inter_area = max(0, min(a[2], b[2]) - max(a[0], b[0])) * max(0, min(a[3], b[3]) - max(a[1], b[1]))
                union_area = (a[2]-a[0])*(a[3]-a[1]) + (b[2]-b[0])*(b[3]-b[1]) - inter_area
                if union_area > 0: ious[i, j] = inter_area / union_area
        return 1 - ious

    def linear_assignment(self, cost_matrix, thresh):
        if cost_matrix.size == 0: return [], list(range(cost_matrix.shape[0])), list(range(cost_matrix.shape[1]))
        row_ind, col_ind = linear_sum_assignment(cost_matrix)
        matches = [(r, c) for r, c in zip(row_ind, col_ind) if cost_matrix[r, c] < thresh]
        u_track = [r for r in range(cost_matrix.shape[0]) if r not in [m[0] for m in matches]]
        u_detection = [c for c in range(cost_matrix.shape[1]) if c not in [m[1] for m in matches]]
        return matches, u_track, u_detection

class KPR_FeatureExtractor:
    def __init__(self, model_path, config_file):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        cfg = build_config(config_file)
        self.model = build_model(cfg, num_classes=1)
        load_pretrained_weights(self.model, model_path)
        self.model.to(self.device)
        self.model.eval()
        self.transform, _ = build_transforms(cfg)

    def __call__(self, img_crops):
        if not isinstance(img_crops, list): img_crops = [img_crops]
        batch = torch.stack([self.transform(crop) for crop in img_crops]).to(self.device)
        with torch.no_grad():
            output = self.model(batch)
            features = output[0] if isinstance(output, tuple) else output
        return features.cpu()

class PlayerReID:
    def __init__(self):
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        config_file = '/content/keypoint_promptable_reidentification/configs/kpr/solider/kpr_occ_duke_test.yaml'
        model_path = '/content/keypoint_promptable_reidentification/pretrained/kpr_solider_occ_duke.pth'
        self.feature_extractor = KPR_FeatureExtractor(model_path, config_file)
        self.similarity_threshold = 0.6

    def get_deep_features(self, patches):
        return self.feature_extractor(patches)

    def process_video(self, video_path, detections_path, tracklets_path, output_path):
        with open(detections_path, 'r') as f: all_detections = json.load(f)
        cap = cv2.VideoCapture(video_path)
        tracker = ByteTrack()
        all_tracklets_data = defaultdict(list)
        y_coords = [d['bbox'][1] for frame in all_detections for d in frame['detections']]
        if not y_coords:
            print("⚠️ No detections found. Skipping bystander filter.")
            y_min, y_max = 0, cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        else:
            y_min, y_max = np.percentile(y_coords, [5, 95])
            print(f"Determined field of play (vertical): {int(y_min)}px to {int(y_max)}px.")

        for frame_data in tqdm(all_detections, desc='Stage 2: Tracking Players'):
            filtered_detections = [d for d in frame_data['detections'] if y_min < (d['bbox'][1] + d['bbox'][3]) / 2 < y_max]
            tracks = tracker.update(filtered_detections)
            for track in tracks: all_tracklets_data[track['track_id']].append({'frame_id': frame_data['frame_id'], 'bbox': track['bbox']})
        with open(tracklets_path, 'w') as f: json.dump(all_tracklets_data, f, indent=2)

        tracklet_features = {}
        cap = cv2.VideoCapture(video_path)
        for track_id, data in tqdm(all_tracklets_data.items(), desc='Stage 3a: Extracting Tracklet Features'):
            if len(data) < 5: continue
            patches_for_tracklet = []
            sample_indices = np.linspace(0, len(data) - 1, min(10, len(data)), dtype=int)
            for i in sample_indices:
                item = data[i]
                cap.set(cv2.CAP_PROP_POS_FRAMES, item['frame_id'])
                ret, frame = cap.read()
                if ret:
                    x1, y1, x2, y2 = [max(0, int(c)) for c in item['bbox']]
                    if x2 > x1 and y2 > y1:
                        patch = frame[y1:y2, x1:x2]
                        patches_for_tracklet.append(patch)
            if patches_for_tracklet:
                features = self.get_deep_features(patches_for_tracklet)
                tracklet_features[track_id] = torch.mean(features, dim=0)
        cap.release()

        track_ids = list(tracklet_features.keys())
        cost_matrix = np.ones((len(track_ids), len(track_ids)))
        for i in range(len(track_ids)):
            for j in range(i + 1, len(track_ids)):
                sim = F.cosine_similarity(tracklet_features[track_ids[i]].unsqueeze(0), tracklet_features[track_ids[j]].unsqueeze(0)).item()
                cost_matrix[i, j] = 1 - sim
                cost_matrix[j, i] = 1 - sim

        id_map = {tid: tid for tid in track_ids}
        visited = set()
        for i in range(len(track_ids)):
            if track_ids[i] in visited: continue
            current_cluster = [track_ids[i]]
            visited.add(track_ids[i])
            for j in range(i + 1, len(track_ids)):
                if cost_matrix[i, j] < (1 - self.similarity_threshold):
                    for member_tid in current_cluster:
                        id_map[track_ids[j]] = id_map[member_tid]
                    current_cluster.append(track_ids[j])
                    visited.add(track_ids[j])

        next_permanent_id = 1
        permanent_id_map = {}
        final_tracks = []
        for frame_idx in range(len(all_detections)):
            frame_players = []
            for track_id, data in all_tracklets_data.items():
                for item in data:
                    if item['frame_id'] == frame_idx:
                        mapped_id = id_map.get(track_id, track_id)
                        if mapped_id not in permanent_id_map:
                            permanent_id_map[mapped_id] = next_permanent_id
                            next_permanent_id += 1
                        permanent_id = permanent_id_map[mapped_id]
                        frame_players.append({'permanent_id': permanent_id, 'bbox': item['bbox']})
            final_tracks.append({'frame_id': frame_idx, 'players': frame_players})

        print(f"✅ Re-ID complete! Identified {next_permanent_id - 1} unique players.")
        with open(output_path, 'w') as f: json.dump(final_tracks, f, indent=2)

        cap = cv2.VideoCapture(video_path)
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        out = cv2.VideoWriter('/content/output/tracking_visualization.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (width, height))
        for frame_data in tqdm(final_tracks, desc="Generating Visualization Video"):
            cap.set(cv2.CAP_PROP_POS_FRAMES, frame_data['frame_id'])
            ret, frame = cap.read()
            if not ret: break
            for player in frame_data['players']:
                pid, bbox = player['permanent_id'], player['bbox']
                x1, y1, x2, y2 = bbox
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f"ID: {pid}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
            out.write(frame)
        cap.release()
        out.release()
        print("✅ Tracking visualization video saved to: /content/output/tracking_visualization.mp4")

class EventDetector:
    def __init__(self):
        self.min_movement_threshold = 10
        self.center_weight = 0.7
        self.movement_weight = 0.2
        self.goal_area_weight = 0.8

    def process_tracks(self, tracks_path, video_width):
        with open(tracks_path, 'r') as f: all_tracks = json.load(f)
        events = []
        player_positions = defaultdict(list)
        for frame_data in tqdm(all_tracks, desc="Stage 4: Detecting Events"):
            frame_id = frame_data['frame_id']
            for player in frame_data['players']:
                pid, bbox = player['permanent_id'], player['bbox']
                center_x = (bbox[0] + bbox[2]) / 2
                center_proximity = 1 - abs(center_x - video_width / 2) / (video_width / 2)
                events.append({'frame_id': frame_id, 'timestamp': frame_id / 30.0, 'event_type': 'center_activity', 'player_id': pid, 'score': self.center_weight * center_proximity, 'bbox': bbox})
                if len(player_positions[pid]) > 0:
                    last_pos = player_positions[pid][-1][1]
                    dist = np.linalg.norm(np.array([center_x, (bbox[1] + bbox[3])/2]) - np.array(last_pos))
                    if dist > self.min_movement_threshold:
                         events.append({'frame_id': frame_id, 'timestamp': frame_id / 30.0, 'event_type': 'movement', 'player_id': pid, 'score': self.movement_weight * (dist / 100), 'bbox': bbox})
                if center_x < video_width * 0.2 or center_x > video_width * 0.8:
                    events.append({'frame_id': frame_id, 'timestamp': frame_id / 30.0, 'event_type': 'goal_area', 'player_id': pid, 'score': self.goal_area_weight, 'bbox': bbox})
                player_positions[pid].append((frame_id, [center_x, (bbox[1] + bbox[3])/2]))
        with open('/content/output/player_events.json', 'w') as f: json.dump(events, f, indent=2)
        print("✅ Event detection complete!")
        return events

class HighlightGenerator:
    def create_reels(self, events, video_path, num_clips=10, clip_duration=5):
        if not events:
            print("No events found to generate highlights.")
            return
        df = pd.DataFrame(events)
        player_scores = df.groupby('player_id')['score'].sum().sort_values(ascending=False)
        top_player = player_scores.index[0]
        player_events = df[df['player_id'] == top_player]
        top_events = player_events.sort_values('score', ascending=False).head(num_clips)
        clip_paths = []
        for i, event in top_events.iterrows():
            start_time = max(0, event['timestamp'] - clip_duration / 2)
            output_clip_path = f"/content/temp_clips/clip_{i}.mp4"
            command = ['ffmpeg', '-ss', str(start_time), '-i', video_path, '-t', str(clip_duration), '-c:v', 'libx264', '-preset', 'fast', '-c:a', 'aac', '-y', output_clip_path]
            subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            clip_paths.append(output_clip_path)
        with open("/content/temp_clips/clips_to_concat.txt", "w") as f:
            for path in clip_paths:
                f.write(f"file '{path}'\n")
        final_reel_path = "/content/output/player_highlights.mp4"
        concat_command = ['ffmpeg', '-f', 'concat', '-safe', '0', '-i', '/content/temp_clips/clips_to_concat.txt', '-c', 'copy', '-y', final_reel_path]
        subprocess.run(concat_command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"Highlight reel created with {len(clip_paths)} clips")
        print(f"✅ Highlight reel saved to: {final_reel_path}")

print("All classes and functions defined. Ready to run the pipeline.")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


ValueError: mutable default <class 'pandas.core.frame.DataFrame'> for field image_gt is not allowed: use default_factory

In [5]:
# @title 4. Stage 1: Player Detection
if 'video_path' in locals() and video_path:
    detector = SoccerPlayerDetector()
    detections_path = '/content/output/detections.json'
    all_detections = detector.process_video(video_path, detections_path)
    print("✅ Detection complete!")
else:
    print("⚠️ Please upload a video in Cell 2 before running this cell.")

⚠️ Please upload a video in Cell 2 before running this cell.


In [6]:
# @title 5. Stage 2 & 3: Tracking and Re-Identification
if 'detections_path' in locals() and os.path.exists(detections_path):
    reid = PlayerReID()
    tracklets_path = '/content/output/tracklets.json'
    final_tracks_path = '/content/output/long_player_track.json'
    reid.process_video(video_path, detections_path, tracklets_path, final_tracks_path)
else:
    print("⚠️ Please run the detection in Cell 4 before running this cell.")

⚠️ Please run the detection in Cell 4 before running this cell.


In [7]:
# @title 6. Stage 4: Event Detection
if 'final_tracks_path' in locals() and os.path.exists(final_tracks_path):
    event_detector = EventDetector()
    cap = cv2.VideoCapture(video_path)
    video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    cap.release()
    events = event_detector.process_tracks(final_tracks_path, video_width)
else:
    print("⚠️ Please run the tracking and Re-ID in Cell 5 before running this cell.")

⚠️ Please run the tracking and Re-ID in Cell 5 before running this cell.


In [8]:
# @title 7. Stage 5: Highlight Reel Generation
if 'events' in locals() and events:
    highlight_generator = HighlightGenerator()
    highlight_generator.create_reels(events, video_path)
else:
    print("⚠️ Please run event detection in Cell 6 before running this cell.")

⚠️ Please run event detection in Cell 6 before running this cell.
